In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import random

# Simulate Sensor Data
def generate_sensor_data(n):
    """Generate simulated data for ground sound, vibration, and temperature."""
    np.random.seed(42)
    sound = np.random.normal(50, 10, n)  # Normal sound levels with some noise
    vibration = np.random.normal(30, 5, n)  # Normal vibration levels
    temperature = np.random.normal(20, 2, n)  # Normal ground temperature

    # Introduce anomalies
    for i in random.sample(range(n), int(n * 0.1)):  # 10% anomalies
        sound[i] += np.random.uniform(20, 50)
        vibration[i] += np.random.uniform(10, 30)
        temperature[i] += np.random.uniform(5, 10)

    return pd.DataFrame({
        'sound': sound,
        'vibration': vibration,
        'temperature': temperature
    })

# Leak Detection
def detect_leaks(data):
    """Detect leaks based on sensor thresholds."""
    leak_indices = data[(data['sound'] > 70) & (data['vibration'] > 40) & (data['temperature'] > 25)].index
    data['leak_detected'] = 0
    data.loc[leak_indices, 'leak_detected'] = 1
    return data

# Adjust Water Distribution
def adjust_water_distribution(data):
    """Simulate water flow adjustments based on leak detection."""
    data['water_flow'] = 100  # Default flow level
    data.loc[data['leak_detected'] == 1, 'water_flow'] = 50  # Reduce flow by 50% in case of leaks
    return data

# Predictive Maintenance
def predictive_maintenance(logs):
    """Train a predictive maintenance model using maintenance logs."""
    logs['days_since_last_repair'] = (logs['date'] - logs['last_repair']).dt.days
    X = logs[['days_since_last_repair']]
    y = logs['repair_needed']

    model = LinearRegression()
    model.fit(X, y)

    # Predict future maintenance needs
    future_days = np.array([[i] for i in range(0, 365, 30)])  # Predict for the next year in 30-day intervals
    predictions = model.predict(future_days)

    return future_days.flatten(), predictions

# Simulate System
if __name__ == "__main__":
    # Generate sensor data
    sensor_data = generate_sensor_data(1000)

    # Detect leaks
    sensor_data = detect_leaks(sensor_data)

    # Adjust water distribution
    sensor_data = adjust_water_distribution(sensor_data)

    # Simulate maintenance logs
    maintenance_logs = pd.DataFrame({
        'date': pd.date_range(start='2023-01-01', periods=10, freq='M'),
        'last_repair': pd.date_range(start='2022-01-01', periods=10, freq='M'),
        'repair_needed': [1, 0, 1, 1, 0, 0, 1, 0, 1, 1]  # Binary target: 1 = repair needed, 0 = no repair
    })

    # Predict future maintenance needs
    future_days, predictions = predictive_maintenance(maintenance_logs)

    # Visualization
    plt.figure(figsize=(10, 6))
    plt.plot(future_days, predictions, label='Predicted Maintenance Need', marker='o')
    plt.title('Predictive Maintenance Over Time')
    plt.xlabel('Days Since Last Repair')
    plt.ylabel('Maintenance Need (Probability)')
    plt.legend()
    plt.grid()
    plt.show()

    print("Leak Detection Results:")
    print(sensor_data[sensor_data['leak_detected'] == 1])
